In [44]:
import nltk
import random
import string
import re, unicodedata
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
import wikipedia as wk
from collections import defaultdict
import warnings
warnings.filterwarnings("ignore")
nltk.download('punkt')
nltk.download('wordnet')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /home/annie/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/annie/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/annie/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [45]:
data=open('HR.txt','r',errors='ignore')
raw=data.read()
raw=raw.lower()

In [46]:
raw[:100]

'human resource analytics is at the intersection of three bodies of knowledge:\n\nhuman resource manage'

In [47]:
sent_tokens=nltk.sent_tokenize(raw)
sent_tokens[:10]

['human resource analytics is at the intersection of three bodies of knowledge:\n\nhuman resource management: sets the meaning and purpose of the analytics.',
 'data warehousing: knowing how to process and store hr data efficiently, automation of  collection of data and cleaning data.',
 'statistical analysis, presentation and interpretation : helps in translating the identified hr  issues into appropriate analyses and communication of results.',
 '5 fundamental principles of analytics\n\nhr analytics is about metrics and measurement.',
 'good metrics definitions, both narrative and formulaic, and their documentation are key.',
 'a professional and good hr analytics person will have the above bodies of knowledge and know their process and intersection.',
 'good communication and collaborative skills are essential.',
 'the in-depth expertise in your organization is likely to exist in hrm.',
 'it.',
 'and decision support.']

In [48]:
def Normalize(text):
    remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
    #word tokenization
    word_token = nltk.word_tokenize(text.lower().translate(remove_punct_dict))
    
    #remove ascii
    new_words = []
    for word in word_token:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    
    #Remove tags
    rmv = []
    for w in new_words:
        text=re.sub("&lt;/?.*?&gt;","&lt;&gt;",w)
        rmv.append(text)
        
    #pos tagging and lemmatization
    tag_map = defaultdict(lambda : wn.NOUN)
    tag_map['J'] = wn.ADJ
    tag_map['V'] = wn.VERB
    tag_map['R'] = wn.ADV
    lmtzr = WordNetLemmatizer()
    lemma_list = []
    rmv = [i for i in rmv if i]
    for token, tag in nltk.pos_tag(rmv):
        lemma = lmtzr.lemmatize(token, tag_map[tag[0]])
        lemma_list.append(lemma)
    return lemma_list    

In [49]:
welcome_input=("hello","hi","greetings","sup","what's up","hey",)
welcome_response=["hi","hey","*nods*","*tips my hat to you*", "I'm glad!"]
def welcome(user_response):
    for word in user_response.split():
        if word.lower in welcome_input:
            return random.choice(welcome_response)

In [50]:
from sklearn.feature_extraction.text import TfidfVectorizer
def generateResponse(user_response):
    robo_response=''
    sent_tokens.append(user_response)
    TfidfVec = TfidfVectorizer(tokenizer=Normalize, stop_words='english')
    tfidf = TfidfVec.fit_transform(sent_tokens)
    vals = linear_kernel(tfidf[-1], tfidf)
    idx=vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    if(req_tfidf==0) or "tell me about" in user_response:
        print("Checking Wikipedia")
        if user_response:
            robo_response = wikipedia_data(user_response)
            return robo_response
    else:
        robo_response = robo_response+sent_tokens[idx]
        return robo_response
def wikipedia_data(input):
    reg_ex = re.search('tell me about (.*)', input)
    try:
        if reg_ex:
            topic = reg_ex.group(1)
            wiki = wk.summary(topic, sentences = 3)
            return wiki
    except Exception as e:
            print("No content has been found")

    

In [ ]:
flag=True
print("My name is Chatter and I'm a chatbot")
while(flag==True):
    user_response = input()
    user_response=user_response.lower()
    if(user_response not in ['bye','shutdown','exit', 'quit']):
        if(user_response=='thanks' or user_response=='thank you' ):
            flag=False
            print("Chatterbot : You are welcome..")
        else:
            if(welcome(user_response)!=None):
                print("Chatterbot : "+welcome(user_response))
            else:
                print("Chatterbot : ",end="")
                print(generateResponse(user_response))
                sent_tokens.remove(user_response)
    else:
        flag=False
        print("Chatterbot : Bye!!! ")

My name is Chatter and I'm a chatbot


 tell me about Mars


Chatterbot : Checking Wikipedia
No content has been found
None


 what is HR?


Chatterbot : hr metrics: application of formulas for measuring and calculating core hr issues so as to draw exact hr results and current scenario of organisation.


...

Chatterbot : Checking Wikipedia
None


 SOlar system


Chatterbot : Checking Wikipedia
None


 solar system


Chatterbot : Checking Wikipedia
None
